# LCEL 인터페이스

사용자 정의 체인을 가능한 쉽게 만들 수 있도록, `Runnable` 프로토콜을 구현

Runnable 프로토콜은 대부분의 컴포넌트에 구현되어있다.

이는 표준 인터페이스로, 사용자 정의 체인을 정의하고 표준 방식으로 호출하는 것을 쉽게 만든다.
표준 인터페이스에는 다음이 포함 :

- `stream` : 응답의 청크를 스트리밍
- `invoke` : 입력에 대해 체인을 호출
- `batch` : 입력 목록에 대한 체인 호출

비동기 메소드 :

- `astream` : 비동기적 응답의 청크를 스트리밍
- `ainvoke` : 비동기적 입력에 대한 체인 호출
- `abatch` : 비동기적 입력 목록에 대해 체인 호출
- `astram_log` : 최종 응답 뿐 아니라 발생하는 중간 단계 스트리밍


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정파일
from dotenv import load_dotenv

# API 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("test")

LangSmith 추적을 시작합니다.
[프로젝트명]
test


LCEL 문법을 사용하여 chain 생성


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 모델 인스턴스화
model = ChatOpenAI()

# 주어진 토픽에 대한 농담을 요청하는 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template("{topic}에 대하여 3문장으로 설명해줘")

# 프롬프트와 모델을 연결 하여 대화체인 생성
chain = prompt | model | StrOutputParser()

## stream : 실시간 출력

이 함수는 `chain.stream` 메서드를 사용하여 주어진 토필에 대해 데이터 스트림을 생성 후 스트림을 반복하여 각 데이터의 내용 (`content`)을 즉시 출력.
`end=""` 인자는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True` 인자는 출력 버퍼를 즉시 비우도록 함.


In [11]:
# chain.stream 메서드를 사용하여 '멀티모달' 토픽에 대한 스트림을 생성하고 반복
for token in chain.stream({"topic": "멀티모달"}):
    # 스트림에서 받은 데이터의 내용 출력 . 줄바꿈 없이 이어서 출력, 버퍼는 지움
    print(token, end="", flush=True)

멀티모달은 여러 가지 다양한 형태의 정보를 동시에 제공하거나 처리하는 기술이다. 이는 텍스트, 음성, 이미지, 동영상 등 다양한 형태의 정보를 통합적으로 활용하여 사용자에게 풍부한 경험을 제공한다. 멀티모달은 사용자의 편의성과 효율성을 높이며, 정보의 이해와 전달을 더욱 원활하게 도와준다.

## invoke: 호출
chain 객체의 invoke 메서드는 주제를 인자로 받아 해당 주제에 대한 처리를 수행합니다.

In [5]:
# chain 객체의 invoke 메서드를 호출하고, 'ChatGPT'라는 주제로 딕셔너리를 전달합니다.
chain.invoke({"topic": "ChatGPT"})

'ChatGPT는 인공지능 챗봇으로 자연어 처리 기술을 이용해 대화를 진행합니다. 사용자의 질문에 대답하고 대화를 이어나가며 유용한 정보를 제공합니다. ChatGPT는 다양한 주제에 대해 대화를 나누는 데 활용됩니다.'

## batch: 배치(단위 실행)
함수 chain.batch는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 topic 키의 값을 사용하여 일괄 처리를 수행합니다.

In [6]:
# 주어진 토픽 리스트를 batch 처리하는 함수 호출
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

['ChatGPT는 인공 지능 챗봇으로 자연어 처리 기술을 사용하여 대화를 수행합니다. 사용자와의 상호작용을 통해 정보를 제공하고 질문에 답변하며 다양한 주제에 대해 대화할 수 있습니다. ChatGPT는 학습을 통해 사용자의 요구에 맞는 정확한 대답을 제공하고 지속적으로 발전하고 있습니다.',
 'Instagram은 사진과 동영상을 공유하고 소셜 네트워크를 통해 다른 사람들과 소통할 수 있는 앱이다. 사용자는 팔로우, 좋아요, 댓글 등을 통해 다른 사용자들과 상호작용할 수 있다. 또한 스토리, 라이브 방송, 필터 등 다양한 기능을 제공한다.']

```max_concurrency``` 매개변수를 사용하여 동시 요청 수를 설정할 수 있습니다

```config``` 딕셔너리는 ```max_concurrency``` 키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정합니다. 여기서는 최대 3개의 작업을 동시에 처리하도록 설정되어 있습니다.

In [7]:
chain.batch(
    [
        {"topic":"ChatGPT"},
        {"topic":"Instagram"},
        {"topic":"BTS"},
        {"topic":"코딩 고수되는 법"},
        {"topic":"인플루언서가 되는 법"}
    ],
    config = {'max_concurrency':3}
)

['ChatGPT는 자연어 처리 기술을 기반으로 한 대화형 인공지능 모델로, 다양한 주제에 대한 대화를 지원합니다. 사용자의 입력을 이해하고 적합한 응답을 생성하여 자연스러운 대화를 제공합니다. ChatGPT를 통해 사용자는 쉽게 정보를 얻고, 질문에 답변을 받을 수 있습니다.',
 'Instagram은 사진과 동영상을 공유하는 소셜 미디어 플랫폼으로, 사용자들은 자신의 일상을 쉽게 공유하고 소통할 수 있습니다. 해시태그를 통해 관심사나 주제별로 소통하는 커뮤니티가 형성되며, 다양한 필터와 편집 기능을 통해 사진을 더 아름답게 보여줄 수 있습니다. 인플루언서들의 활동이 활발하고 시각적인 콘텐츠가 주를 이루는 특징이 있습니다.',
 "BTS는 한국의 7인조 보이 그룹으로, 그들의 음악은 전 세계적으로 인기를 끌고 있다. 뛰어난 댄스 실력과 감각적인 음악으로 팬들을 매료시키고 있으며, 다양한 사회 문제에 대한 메시지를 담은 가사로도 유명하다. 그들은 K-pop 산업을 세계적인 수준으로 끌어올리는 데 큰 역할을 하고 있으며, 'ARMY'라 불리는 열렬한 팬들로 인해 전 세계적으로 큰 사랑을 받고 있다.",
 '1. 많은 연습과 경험을 통해 코딩 실력을 향상시키고, 다양한 프로젝트에 참여하여 실무 경험을 쌓는 것이 중요하다.\n2. 지속적인 학습과 자기계발을 통해 최신 기술과 도구에 대한 이해를 높이고, 문제 해결능력을 향상시키는 것이 필요하다.\n3. 다른 개발자들과의 협업 및 소통을 통해 다양한 시각과 아이디어를 수용하고 발전시키며, 지속적인 성장을 추구해야 한다.',
 '1. 자신만의 독특한 콘텐츠와 스타일을 갖추어 다른 사람들에게 영감을 주는 것이 가장 중요하다.\n2. 소통 능력을 갖추어 팔로워들과 소통하고 상호작용하는 것이 인플루언서로 성공하는 데 중요하다.\n3. 꾸준히 업데이트하고 새로운 트렌드를 빠르게 반영하여 팔로워들의 관심을 유지하는 것도 중요하다.']

## async stream: 비동기 스트림
함수 ```chain.astream```은 비동기 스트림을 생성하며, 주어진 토픽에 대한 메시지를 비동기적으로 처리합니다.

비동기 for 루프(```async for```)를 사용하여 스트림에서 메시지를 순차적으로 받아오고, ```print``` 함수를 통해 메시지의 내용(```s.content```)을 즉시 출력합니다. ```end=""```는 출력 후 줄바꿈을 하지 않도록 설정하며, ```flush=True```는 출력 버퍼를 강제로 비워 즉시 출력되도록 합니다.

In [13]:
# 비동기 스트림을 사용하여 'YouTube' 토픽의 메시지를 처리합니다.
async for token in chain.astream({"topic": "YouTube"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(token, end="", flush=True)

YouTube는 동영상을 공유하고 시청할 수 있는 온라인 비디오 플랫폼이다. 사용자들은 다양한 주제의 동영상을 업로드하고 시청할 수 있으며, 구독 기능을 통해 즐겨찾는 채널을 구독할 수 있다. 또한 광고 수익을 얻을 수 있는 YouTube 파트너 프로그램을 통해 수익을 창출할 수 있다.

## async invoke: 비동기 호출
```chain``` 객체의 ```ainvoke``` 메서드는 비동기적으로 주어진 인자를 사용하여 작업을 수행합니다. 여기서는 ```topic```이라는 키와 NVDA(엔비디아의 티커) 라는 값을 가진 딕셔너리를 인자로 전달하고 있습니다. 이 메서드는 특정 토픽에 대한 처리를 비동기적으로 요청하는 데 사용될 수 있습니다.

In [14]:
# 비동기 체인 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽 처리
my_process = chain.ainvoke({"topic":"NVDA"})

In [15]:
await my_process

'NVDA는 전 세계적으로 유명한 시각 장애인을 위한 무료 스크린 리더 소프트웨어이다. 이 소프트웨어는 텍스트를 음성으로 변환하여 시각 장애인이 컴퓨터를 사용할 수 있도록 도와준다. NVDA는 다양한 운영 체제에서 작동하며, 많은 사용자들에게 편리한 서비스를 제공하고 있다.'

In [16]:
await chain.ainvoke({"topic":"NVDA"})

'NVDA는 미국의 그래픽 카드 제조사인 엔비디아의 주식 코드이다. 엔비디아는 주로 게임 그래픽 카드와 인공지능을 위한 그래픽 처리 장치를 생산하는 기업으로, NVDA 주식은 기술 기업 주식 중에서 인기가 높은 편이다. 엔비디아의 기술력과 성장 가능성으로 인해 NVDA 주식은 투자자들 사이에서 인기가 높은 주식 중 하나이다.'

## async batch: 비동기 배치
함수 ```abatch```는 비동기적으로 일련의 작업을 일괄 처리합니다.

이 예시에서는 ```chain``` 객체의 ```abatch``` 메서드를 사용하여 ```topic``` 에 대한 작업을 비동기적으로 처리하고 있습니다.

```await``` 키워드는 해당 비동기 작업이 완료될 때까지 기다리는 데 사용됩니다.

In [21]:

# 주어진 토픽에 대해 비동기적으로 일괄 처리를 수행합니다.
my_abatch_process = chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)

In [22]:
await my_abatch_process

['YouTube는 구글이 소유하고 있는 동영상 공유 플랫폼으로, 사용자들은 영상을 업로드하고 시청할 수 있습니다. 다양한 콘텐츠가 제공되며, 사용자들은 구독과 좋아요, 댓글 등을 통해 소통할 수 있습니다. 광고수익을 얻을 수 있는 유튜버도 많이 활동하고 있습니다.',
 'Instagram은 사진과 동영상을 공유하고 다른 사람들과 소통하는 소셜 미디어 플랫폼이다. 셀피나 일상 생활을 담은 포스트를 올리고 팔로워들과 소통하며 새로운 인연을 만들 수 있다. 해시태그를 통해 관심사나 주제별로 다양한 콘텐츠를 찾아보고 공유할 수 있다.',
 'Facebook은 세계적으로 가장 인기 있는 소셜 네트워킹 서비스로 사용자들이 친구, 가족과 소통하고 콘텐츠를 공유할 수 있는 플랫폼입니다. 사용자들은 텍스트, 사진, 동영상 등 다양한 형식의 콘텐츠를 공유하며 소셜 미디어 활동을 즐길 수 있습니다. Facebook은 개인정보 보호와 데이터 이용에 대한 논란이 있지만, 여전히 많은 사람들이 활발하게 이용하고 있는 플랫폼입니다.']

# Parallel : 병렬성

Langchain Expression Language가 병렬 요청을 지원하는 방법을 살펴보자. 예를들어, 
```RunnableParallel```을 사용할 때 각 요소를 병렬로 실행.

```langchain_core.runnables``` 모듈의 ```RunnableParallel``` 클래스를 사용하여 두가지 작업을 병렬로 실행하는 예시이다.

```ChatPromptTemplate.from_template``` 메서드를 사용하여 주어진 ```country``` 에 대한 수도와 면적을 구하는 두개의 체인(```chain1, chain2```)를 만든다.

이 체인들은 각각 ```model``` 과 파이프(```|```) 연산자를 통해 연결됨. 마지막으로 ```RunnableParallel``` 클래스를 사용하여 이 두 체인을 ```capital```과 ```area```라는 키로 결합하여 동시에 실행할 수 있는 ```combined``` 객체를 생성.

In [23]:
from langchain_core.runnables import RunnableParallel

#{country} 의 수도를 물어보는 체인을 생성합니다.
chain1 = (
    PromptTemplate.from_template("{country}의 수도는 어디야?" )
    | model
    | StrOutputParser()   
)

# {country} 면적을 물어보는 체인을 생성
chain2 = (
    PromptTemplate.from_template("{country}의 면적은 얼마나 돼 ?" )
    | model
    | StrOutputParser()   
)

# 위의 2개 체인을 동시에 생성하는 병렬 실행 체인을 생성
combined = RunnableParallel(capital = chain1, area = chain2)

```chain1.invoke()``` 함수는 ```chain1``` 객체의 ```invoke``` 메서드를 호출합니다.

이때, ```country```이라는 키에 대한민국라는 값을 가진 딕셔너리를 인자로 전달합니다.

In [24]:
# chain1 를 실행합니다.
chain1.invoke({"country": "대한민국"})

'대한민국의 수도는 서울이다.'

이번에는 ```chain2.invoke()``` 를 호출합니다. ```country``` 키에 다른 국가인 미국 을 전달합니다.

In [25]:
# chain2 를 실행합니다.
chain2.invoke({"country": "미국"})

'미국의 면적은 약 9,826,675 km² 입니다.'

```combined``` 객체의 ```invoke``` 메서드는 주어진 ```country```에 대한 처리를 수행합니다.

이 예제에서는 대한민국라는 주제를 ```invoke``` 메서드에 전달하여 실행합니다.

In [26]:
# 병렬 실행 체인을 실행합니다.
combined.invoke({"country": "대한민국"})

{'capital': '대한민국의 수도는 서울입니다.', 'area': '대한민국의 면적은 약 100,363.4㎢ 입니다.'}

# 배치에서의 병렬 처리
병렬 처리는 다른 실행 가능한 코드와 결합될 수 있습니다. 배치와 병렬 처리를 사용해 보도록 합시다.

```chain1.batch``` 함수는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 "topic" 키에 해당하는 값을 처리합니다. 이 예시에서는 "대한민국"와 "미국"라는 두 개의 토픽을 배치 처리하고 있습니다.

In [27]:
# 배치 처리를 수행합니다.
chain1.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 수도는 서울이야.', '미국의 수도는 워싱턴 D.C.입니다.']

```chain2```.batch 함수는 여러 개의 딕셔너리를 리스트 형태로 받아, 일괄 처리(```batch```)를 수행합니다.

이 예시에서는 대한민국와 미국라는 두 가지 국가에 대한 처리를 요청합니다.

In [28]:
# 배치 처리를 수행합니다.
chain2.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 총 면적은 약 100,363.4km² 입니다.', '미국의 면적은 약 9,833,520km² 입니다.']

```combined.batch``` 함수는 주어진 데이터를 배치로 처리하는 데 사용됩니다. 이 예시에서는 두 개의 딕셔너리 객체를 포함하는 리스트를 인자로 받아 각각 대한민국와 미국 두 나라에 대한 데이터를 배치 처리합니다.

In [29]:
# 주어진 데이터를 배치로 처리합니다.
combined.batch([{"country": "대한민국"}, {"country": "미국"}])

[{'capital': '대한민국의 수도는 서울이다.', 'area': '대한민국의 면적은 약 100,363 제곱 킬로미터입니다.'},
 {'capital': '미국의 수도는 워싱턴 D.C.입니다.', 'area': '미국의 총 면적은 대략 9,833,520km²입니다.'}]